### CNN on CIFR Assignment:

1.  Please visit this link to access the state-of-art DenseNet code for reference - DenseNet - cifar10 notebook link
2.  You need to create a copy of this and "retrain" this model to achieve 90+ test accuracy. 
3.  You cannot use DropOut layers.
4.  You MUST use Image Augmentation Techniques.
5.  You cannot use an already trained model as a beginning points, you have to initilize as your own
6.  You cannot run the program for more than 300 Epochs, and it should be clear from your log, that you have only used 300 Epochs
7.  You cannot use test images for training the model.
8.  You cannot change the general architecture of DenseNet (which means you must use Dense Block, Transition and Output blocks as mentioned in the code)
9.  You are free to change Convolution types (e.g. from 3x3 normal convolution to Depthwise Separable, etc)
10. You cannot have more than 1 Million parameters in total
11. You are free to move the code from Keras to Tensorflow, Pytorch, MXNET etc. 
12. You can use any optimization algorithm you need. 
13. You can checkpoint your model and retrain the model from that checkpoint so that no need of training the model from first if you lost at any epoch while training. You can directly load that model and Train from that epoch. 

In [14]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 5197198190959437454
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 14509932544
locality {
  bus_id: 1
  links {
  }
}
incarnation: 18187921042933008933
physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"
]


In [15]:
import tensorflow as tf
from tensorflow.keras import models, layers
from tensorflow.keras.models import Model
from tensorflow.keras.layers import BatchNormalization, Activation, Flatten
from tensorflow.keras.optimizers import Adam,RMSprop

In [16]:
batch_size = 128
num_classes = 10
epochs = 100
l = 12
num_filter = 24
compression = 0.5
dropout_rate = 0.2

In [17]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()
img_height, img_width, channel = X_train.shape[1],X_train.shape[2],X_train.shape[3]

y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes) 

In [18]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(50000, 32, 32, 3)
(10000, 32, 32, 3)
(50000, 10)
(10000, 10)


In [19]:
def denseblock(input, num_filter = 12,dropout_rate = 0.2):
    global compression
    temp = input
    for _ in range(l): 
        BatchNorm = layers.BatchNormalization()(temp)
        relu = layers.Activation('relu')(BatchNorm)
        Conv2D_3_3 = layers.Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        concat = layers.Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

## transition Blosck
def transition(input, num_filter = 12,dropout_rate = 0.2):
    global compression
    BatchNorm = layers.BatchNormalization()(input)
    relu = layers.Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = layers.Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    avg = layers.AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    return avg

#output layer
def output_layer(input):
    global compression
    BatchNorm = layers.BatchNormalization()(input)
    relu = layers.Activation('relu')(BatchNorm)
    AvgPooling = layers.AveragePooling2D(pool_size=(2,2))(relu)
    flat = layers.Flatten()(AvgPooling)
    output = layers.Dense(num_classes, activation='softmax')(flat)
    return output

In [20]:
def lr_schedule(epoch):
    lr = 1e-3
    if epoch > 90:
        lr *= 1e-3
    elif epoch > 70:
        lr *= 1e-2
    elif epoch > 50:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr

In [21]:
import os
from keras.callbacks import ModelCheckpoint, CSVLogger
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'cifar10_densenet_model.{epoch:02d}.h5'
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
filepath = os.path.join(save_dir, model_name)

In [22]:
checkpoint = ModelCheckpoint(filepath=filepath,monitor='val_acc',verbose=1,save_best_only=True)

In [23]:
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
    rotation_range=5,
    width_shift_range=0.1,
    height_shift_range=0.5,
    horizontal_flip=True,
    shear_range = 5
    )
datagen.fit(X_train)

In [24]:
input = layers.Input(shape=(img_height, img_width, channel,))
First_Conv2D = layers.Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

First_Block = denseblock(First_Conv2D, num_filter)
First_Transition = transition(First_Block, num_filter)

Second_Block = denseblock(First_Transition, num_filter)
Second_Transition = transition(Second_Block, num_filter)

Third_Block = denseblock(Second_Transition, num_filter)
Third_Transition = transition(Third_Block, num_filter)

Last_Block = denseblock(Third_Transition,  num_filter)
output = output_layer(Last_Block)

In [25]:
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import LearningRateScheduler
import numpy as np
model = Model(inputs=[input], outputs=[output])
model.compile(loss='categorical_crossentropy',optimizer=RMSprop(1e-3),metrics=['accuracy'])
lr_scheduler = LearningRateScheduler(lr_schedule)

lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),cooldown=0,patience=5,min_lr=0.5e-6)
model.summary()

callbacks = [ lr_reducer, lr_scheduler,checkpoint,]

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv2d_64 (Conv2D)              (None, 32, 32, 24)   648         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_64 (BatchNo (None, 32, 32, 24)   96          conv2d_64[0][0]                  
__________________________________________________________________________________________________
activation_64 (Activation)      (None, 32, 32, 24)   0           batch_normalization_64[0][0]     
____________________________________________________________________________________________

In [26]:
model.fit(datagen.flow(X_train, y_train, batch_size=batch_size),\
                    steps_per_epoch=X_train.shape[0] // batch_size,epochs=epochs,\
                    verbose=1,validation_data=(X_test,y_test),callbacks=callbacks)

Epoch 1/100
Learning rate:  0.001
390/390 [==============================] - 113s 266ms/step - loss: 1.7718 - accuracy: 0.3449 - val_loss: 2.2789 - val_accuracy: 0.2918
Epoch 2/100
Learning rate:  0.001
390/390 [==============================] - 98s 250ms/step - loss: 1.3689 - accuracy: 0.5023 - val_loss: 2.1380 - val_accuracy: 0.4113
Epoch 3/100
Learning rate:  0.001
390/390 [==============================] - 97s 249ms/step - loss: 1.1578 - accuracy: 0.5871 - val_loss: 1.6271 - val_accuracy: 0.5265
Epoch 4/100
Learning rate:  0.001
390/390 [==============================] - 97s 248ms/step - loss: 1.0266 - accuracy: 0.6367 - val_loss: 1.7233 - val_accuracy: 0.5083
Epoch 5/100
Learning rate:  0.001
390/390 [==============================] - 97s 250ms/step - loss: 0.9408 - accuracy: 0.6658 - val_loss: 1.1911 - val_accuracy: 0.6315
Epoch 6/100
Learning rate:  0.001
390/390 [==============================] - 97s 250ms/step - loss: 0.8732 - accuracy: 0.6919 - val_loss: 1.0492 - val_accuracy

In [27]:
model.evaluate(X_test, y_test)

313/313 [==============================] - 6s 18ms/step - loss: 0.3238 - accuracy: 0.9094


[0.32375726103782654, 0.9093999862670898]